In [3]:
import os
import pandas as pd

root_dir = "/Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI"
all_dataframes = []

def process_csv(file_path, user, session, task, relative_name):
    try:
        df = pd.read_csv(file_path)
        if df.empty:
            return None

        df.dropna(inplace=True)

        # Normalize timestamp
        timestamp_col = df.columns[0]
        df.rename(columns={timestamp_col: 'timestamp'}, inplace=True)
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        df.dropna(subset=['timestamp'], inplace=True)
        df.set_index('timestamp', inplace=True)

        # Prefix column names
        base_name = relative_name.replace(".csv", "").replace(os.sep, "_")
        full_prefix = f"{task}_{base_name}" if task else base_name
        df.columns = [f"{full_prefix}_{col}" for col in df.columns]

        # Add user and session columns
        df['user'] = user
        df['session'] = session
        return df
    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")
        return None

# Walk through user/session/task folders
for user in os.listdir(root_dir):
    user_path = os.path.join(root_dir, user)
    if not os.path.isdir(user_path):
        continue

    for session in os.listdir(user_path):
        session_path = os.path.join(user_path, session)
        if not os.path.isdir(session_path) or not session.lower().startswith('sesion'):
            continue

        # Top-level CSVs in session
        for file in os.listdir(session_path):
            full_path = os.path.join(session_path, file)
            if file.endswith(".csv") and os.path.isfile(full_path):
                df = process_csv(full_path, user, session, task=None, relative_name=file)
                if df is not None:
                    all_dataframes.append(df)

        # Task-level nested files
        for task in os.listdir(session_path):
            task_path = os.path.join(session_path, task)
            if not os.path.isdir(task_path) or task.lower().startswith("sensor") or task.endswith(".csv"):
                continue

            for root_dir_path, _, files in os.walk(task_path):
                for file in files:
                    if file.endswith(".csv"):
                        full_path = os.path.join(root_dir_path, file)
                        rel_path = os.path.relpath(full_path, task_path)
                        rel_name = rel_path.replace(os.sep, "_")
                        df = process_csv(full_path, user, session, task=task, relative_name=rel_name)
                        if df is not None:
                            all_dataframes.append(df)

# Merge and reorder
if all_dataframes:
    print(f"📦 Merging {len(all_dataframes)} files...")
    merged_df = pd.concat(all_dataframes, axis=0)
    merged_df.reset_index(inplace=True)

    # Group by timestamp, user, session
    merged_df = merged_df.groupby(['user', 'session', 'timestamp'], as_index=False).first()

    # Move 'user', 'session', 'timestamp' to front
    cols = merged_df.columns.tolist()
    ordered_cols = ['user', 'session', 'timestamp'] + [col for col in cols if col not in ['user', 'session', 'timestamp']]
    merged_df = merged_df[ordered_cols]

    merged_df.to_csv("final_HUMIdb_with_tasks.csv", index=False)
    print("✅ Final CSV saved as: final_HUMIdb_with_tasks.csv")
else:
    print("⚠️ No CSVs processed.")


❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/wifi.csv: Error tokenizing data. C error: Expected 6 fields in line 5, saw 9

❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/FINGER_7/SENSORS/sensor_temp.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/FINGER_7/SENSORS/sensor_humd.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/FINGER_0/SENSORS/sensor_temp.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/FINGER_0/SENSORS/sensor_humd.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/FINGER_9/SENSORS/sensor_temp.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/

/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='c

❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/TOUCH/SENSORS/sensor_humd.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/AIR_X/SENSORS/sensor_temp.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/AIR_X/SENSORS/sensor_humd.csv: No columns to parse from file


/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='c

❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/SCROLLDOWN/SENSORS/sensor_temp.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/SCROLLDOWN/SENSORS/sensor_humd.csv: No columns to parse from file


...

❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/FINGER_7/SENSORS/sensor_temp.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/FINGER_7/SENSORS/sensor_humd.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/FINGER_0/SENSORS/sensor_temp.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/FINGER_0/SENSORS/sensor_humd.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/FINGER_9/SENSORS/sensor_temp.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/FINGER_9/SENSORS/sensor_humd.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2

/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='c

❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/TOUCH/SENSORS/sensor_humd.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/AIR_X/SENSORS/sensor_temp.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/AIR_X/SENSORS/sensor_humd.csv: No columns to parse from file
❌ Error processing /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/567/Sesion2/SCROLLDOWN/SENSORS/sensor_temp.csv: No columns to parse from file


/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_40908/2779392201.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='c

KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd

# ✅ Change this to your HuMI dataset path
root_dir = "/Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI"

all_dataframes = []

def process_csv(file_path, user, session, task, relative_name):
    try:
        if os.path.getsize(file_path) == 0:
            print(f"⚠️ Skipped empty file: {file_path}")
            return None

        # Read CSV while skipping malformed rows
        df = pd.read_csv(file_path, on_bad_lines='skip')

        if df.empty:
            print(f"⚠️ Skipped empty DataFrame after reading: {file_path}")
            return None

        df.dropna(inplace=True)
        if df.empty:
            return None

        # Rename the first column to 'timestamp'
        timestamp_col = df.columns[0]
        df.rename(columns={timestamp_col: 'timestamp'}, inplace=True)

        # Convert timestamp from UNIX milliseconds to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', errors='coerce')
        df.dropna(subset=['timestamp'], inplace=True)
        if df.empty:
            return None

        df.set_index('timestamp', inplace=True)

        # Create prefixed column names
        base_name = relative_name.replace(".csv", "").replace(os.sep, "_")
        full_prefix = f"{task}_{base_name}" if task else base_name
        df.columns = [f"{full_prefix}_{col}" for col in df.columns]

        # Add user and session columns
        df['user'] = user
        df['session'] = session

        return df

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")
        return None

# Walk through each user/session/task and process CSV files
for user in os.listdir(root_dir):
    user_path = os.path.join(root_dir, user)
    if not os.path.isdir(user_path):
        continue

    for session in os.listdir(user_path):
        session_path = os.path.join(user_path, session)
        if not os.path.isdir(session_path) or not session.lower().startswith('sesion'):
            continue

        # Process top-level CSVs in the session folder
        for file in os.listdir(session_path):
            full_path = os.path.join(session_path, file)
            if file.endswith(".csv") and os.path.isfile(full_path):
                df = process_csv(full_path, user, session, task=None, relative_name=file)
                if df is not None:
                    all_dataframes.append(df)

        # Process task folders like AIR_0, FINGER_1, MICRO, etc.
        for task in os.listdir(session_path):
            task_path = os.path.join(session_path, task)
            if not os.path.isdir(task_path) or task.lower().startswith("sensor") or task.endswith(".csv"):
                continue

            for root_dir_path, _, files in os.walk(task_path):
                for file in files:
                    if file.endswith(".csv"):
                        full_path = os.path.join(root_dir_path, file)
                        rel_path = os.path.relpath(full_path, task_path)
                        rel_name = rel_path.replace(os.sep, "_")
                        df = process_csv(full_path, user, session, task=task, relative_name=rel_name)
                        if df is not None:
                            all_dataframes.append(df)

# Merge all cleaned dataframes
if all_dataframes:
    print(f"📦 Merging {len(all_dataframes)} cleaned DataFrames...")
    merged_df = pd.concat(all_dataframes, axis=0)
    merged_df.reset_index(inplace=True)

    # Group by user, session, timestamp to align same moments
    merged_df = merged_df.groupby(['user', 'session', 'timestamp'], as_index=False).first()

    # Reorder columns: user, session, timestamp first
    cols = merged_df.columns.tolist()
    ordered_cols = ['user', 'session', 'timestamp'] + [col for col in cols if col not in ['user', 'session', 'timestamp']]
    merged_df = merged_df[ordered_cols]

    # Save final output
    merged_df.to_csv("final_HUMIdb_with_tasks.csv", index=False)
    print("✅ Final CSV saved as: final_HUMIdb_with_tasks.csv")
else:
    print("⚠️ No usable CSVs found to merge.")


⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/MICRO/swipe.csv
⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/MICRO/SENSORS/sensor_magn.csv
⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/MICRO/SENSORS/sensor_gyro.csv
⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/MICRO/SENSORS/sensor_temp.csv
⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/MICRO/SENSORS/sensor_grav.csv
⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/MICRO/SENSORS/sensor_ligh.csv
⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/MICRO/SENSORS/sensor_prox.csv

/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['ti

⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/TOUCH/SENSORS/sensor_prox.csv
⚠️ Skipped empty file: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/TOUCH/SENSORS/sensor_humd.csv
⚠️ Skipped empty file: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/AIR_X/SENSORS/sensor_temp.csv
⚠️ Skipped empty file: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/AIR_X/SENSORS/sensor_humd.csv


/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['ti

⚠️ Skipped empty file: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/SCROLLDOWN/SENSORS/sensor_temp.csv
⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/SCROLLDOWN/SENSORS/sensor_prox.csv
⚠️ Skipped empty file: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/135/Sesion1/SCROLLDOWN/SENSORS/sensor_humd.csv


...

/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['ti

⚠️ Skipped empty file: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/307/Sesion1/AIR_X/SENSORS/sensor_temp.csv
⚠️ Skipped empty file: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/307/Sesion1/AIR_X/SENSORS/sensor_humd.csv
⚠️ Skipped empty file: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/307/Sesion1/SCROLLDOWN/SENSORS/sensor_temp.csv


/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['ti

⚠️ Skipped empty DataFrame after reading: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/307/Sesion1/SCROLLDOWN/SENSORS/sensor_prox.csv
⚠️ Skipped empty file: /Users/kkartikaggarwal/Desktop/coding folder2/canrabank/HuMI/307/Sesion1/SCROLLDOWN/SENSORS/sensor_humd.csv


/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
/var/folders/3l/sv8msxn916vcnqfh718hz_m40000gn/T/ipykernel_43316/3988669247.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp(ms)'] = pd.to_datetime(df['timestamp(ms)'], errors='coerce')
